In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
dframe= pd.read_csv('C:/Users/Harsh/Desktop/npcl.5_id_data.csv')

In [3]:
id=dframe['location_id'].unique()

In [4]:
d_frame=dframe[['data.grid_reading_kwh','data.grid_kwh_unit','data.creation_time','data.status',
'data.meter_sr_no','data.error_reason','data.R_voltage','data.Y_volatge',
'data.B_voltage', 'data.r_current', 'data.y_current','data.b_current','site_id',
'location_id','data.cumm_pf','data.frequency']]

In [63]:
# Status "P"
combined_df=d_frame[d_frame['data.status']== "P"]

# null values
if combined_df.isnull().sum().sum()>0:
    combined_df.dropna(inplace=True)

# special character
def special_char():
    special_character_pattern = r'[!@#$%^&*()]'
    rows_with_special_chars = combined_df[combined_df['location_id'].str.contains(special_character_pattern, regex=True)]
    return ( rows_with_special_chars)
if combined_df['location_id'].dtype != 'int64':
    special_char()

#  creation time relabelling 
# combined_df['data.creation_time']=pd.to_datetime(combined_df['data.creation_time'])

# integer conversion
combined_df['data.grid_reading_kwh']=combined_df['data.grid_reading_kwh'].round()

In [64]:
final_lst=[]
for each_id in id:
    df=combined_df[combined_df['location_id'] == each_id ]
    df['data.creation_time']=pd.to_datetime(df['data.creation_time'])
    df.set_index(['data.creation_time'],drop=True, inplace=True)
    df.loc[df['data.error_reason'] == 'No Consumption', 'data.grid_kwh_unit'] = 0
    df['units']=round((df['data.grid_reading_kwh']-df['data.grid_reading_kwh'].shift(1)),2)
    df.dropna(inplace=True)
    df = df[df['units'] >= 0]

    # resampling
    
    


    final_lst.append(df)

final_df = pd.concat(final_lst)

In [78]:
df=final_lst[4]

In [79]:
df

,data.grid_reading_kwh,data.grid_kwh_unit,data.status,data.meter_sr_no,data.error_reason,data.R_voltage,data.Y_volatge,data.B_voltage,data.r_current,data.y_current,data.b_current,site_id,location_id,data.cumm_pf,data.frequency,units
data.creation_time,,,,,,,,,,,,,,,,
2021-05-01 00:04:01,460.0,0.0,P,DAL0000997,No Consumption,235.8,240.8,226.6,0.52,7.30,7.31,6075bb51153a20.38235471,500000010032,0.99,50.3,0.0
2021-05-01 00:06:06,460.0,0.0,P,DAL0000997,No Consumption,234.4,238.4,228.9,7.79,7.29,1.11,6075bb51153a20.38235471,500000010032,0.99,50.1,0.0
2021-05-01 00:08:12,460.0,0.0,P,DAL0000997,No Consumption,234.4,238.4,228.9,7.79,7.29,1.11,6075bb51153a20.38235471,500000010032,0.99,50.1,0.0
2021-05-01 00:10:17,461.0,1.0,P,DAL0000997,Success,235.9,236.9,227.0,8.08,7.35,7.57,6075bb51153a20.38235471,500000010032,0.99,50.1,1.0
2021-05-01 00:12:21,461.0,0.0,P,DAL0000997,No Consumption,235.9,236.9,227.0,8.08,7.35,7.57,6075bb51153a20.38235471,500000010032,0.99,50.1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-17 23:17:09,21680.0,1.0,P,DAL0000997,Success,234.8,232.3,224.5,7.83,0.04,1.11,6075bb51153a20.38235471,500000010032,1.00,50.1,1.0
2023-08-17 23:26:52,21680.0,0.0,P,DAL0000997,No Consumption,236.7,233.6,224.6,7.78,0.00,0.47,6075bb51153a20.38235471,500000010032,0.99,50.0,0.0
2023-08-17 23:36:34,21681.0,1.0,P,DAL0000997,Success,237.2,233.7,229.9,7.72,0.00,0.72,6075bb51153a20.38235471,500000010032,0.99,50.2,1.0


In [80]:
df_min=df[['data.grid_reading_kwh']].resample(rule="1H").min()
df_min

,data.grid_reading_kwh
data.creation_time,
2021-05-01 00:00:00,460.0
2021-05-01 01:00:00,463.0
2021-05-01 02:00:00,467.0
2021-05-01 03:00:00,470.0
2021-05-01 04:00:00,473.0
...,...
2023-08-17 19:00:00,21673.0
2023-08-17 20:00:00,21673.0
2023-08-17 21:00:00,21674.0


In [81]:
dfresample=df[['data.grid_reading_kwh']].resample(rule='1H').max()
dfresample

,data.grid_reading_kwh
data.creation_time,
2021-05-01 00:00:00,463.0
2021-05-01 01:00:00,467.0
2021-05-01 02:00:00,470.0
2021-05-01 03:00:00,473.0
2021-05-01 04:00:00,477.0
...,...
2023-08-17 19:00:00,21673.0
2023-08-17 20:00:00,21674.0
2023-08-17 21:00:00,21676.0


In [102]:
dfresample["derived_units"]=dfresample['data.grid_reading_kwh']-dfresample['data.grid_reading_kwh'].shift(1)
dfresample

,data.grid_reading_kwh,derived_units,units
data.creation_time,,,
2021-05-01 00:00:00,463.0,NaN,3.0
2021-05-01 01:00:00,467.0,4.0,4.0
2021-05-01 02:00:00,470.0,3.0,3.0
2021-05-01 03:00:00,473.0,3.0,3.0
2021-05-01 04:00:00,477.0,4.0,4.0
...,...,...,...
2023-08-17 19:00:00,21673.0,0.0,0.0
2023-08-17 20:00:00,21674.0,1.0,1.0
2023-08-17 21:00:00,21676.0,2.0,2.0


In [103]:
dfresample.isnull().sum()

data.grid_reading_kwh    163
derived_units            195
units                      0
dtype: int64

In [86]:
dfresample['units']=df_resample['units']

In [87]:
dfresample

,data.grid_reading_kwh,derived_units,units
data.creation_time,,,
2021-05-01 00:00:00,463.0,3.0,3.0
2021-05-01 01:00:00,467.0,4.0,4.0
2021-05-01 02:00:00,470.0,3.0,3.0
2021-05-01 03:00:00,473.0,3.0,3.0
2021-05-01 04:00:00,477.0,4.0,4.0
...,...,...,...
2023-08-17 19:00:00,21673.0,0.0,0.0
2023-08-17 20:00:00,21674.0,1.0,1.0
2023-08-17 21:00:00,21676.0,2.0,2.0


In [89]:
indices_with_missing_values = dfresample[dfresample.isnull().any(axis=1)].index

# Display the indices
print(indices_with_missing_values)

DatetimeIndex(['2021-05-24 17:00:00', '2021-05-24 18:00:00',
               '2021-05-24 19:00:00', '2021-05-24 20:00:00',
               '2021-05-24 21:00:00', '2021-05-24 22:00:00',
               '2021-05-24 23:00:00', '2021-05-25 00:00:00',
               '2021-05-25 01:00:00', '2021-05-25 02:00:00',
               ...
               '2023-05-13 22:00:00', '2023-05-15 08:00:00',
               '2023-05-15 09:00:00', '2023-05-15 10:00:00',
               '2023-05-17 09:00:00', '2023-05-17 10:00:00',
               '2023-05-17 11:00:00', '2023-05-17 12:00:00',
               '2023-06-22 16:00:00', '2023-07-13 03:00:00'],
              dtype='datetime64[ns]', name='data.creation_time', length=163, freq=None)


In [91]:
dfresample['2021-05-24 10:00:00':'2021-05-25 12:00:00']

,data.grid_reading_kwh,derived_units,units
data.creation_time,,,
2021-05-24 10:00:00,1535.0,0.0,0.0
2021-05-24 11:00:00,1535.0,0.0,0.0
2021-05-24 12:00:00,1537.0,1.0,2.0
2021-05-24 13:00:00,1537.0,0.0,0.0
2021-05-24 14:00:00,1537.0,0.0,0.0
2021-05-24 15:00:00,1538.0,1.0,1.0
2021-05-24 16:00:00,1539.0,1.0,1.0
2021-05-24 17:00:00,NaN,NaN,0.0
2021-05-24 18:00:00,NaN,NaN,0.0


In [104]:
dfres=dfresample[['data.grid_reading_kwh']].resample('1H').bfill()

In [105]:
dfres['2021-05-24 10:00:00':'2021-05-25 12:00:00']

,data.grid_reading_kwh
data.creation_time,
2021-05-24 10:00:00,1535.0
2021-05-24 11:00:00,1535.0
2021-05-24 12:00:00,1537.0
2021-05-24 13:00:00,1537.0
2021-05-24 14:00:00,1537.0
2021-05-24 15:00:00,1538.0
2021-05-24 16:00:00,1539.0
2021-05-24 17:00:00,NaN
2021-05-24 18:00:00,NaN


In [ ]:
dfresample=dfresample.resample(rule='1H').agg({'column_name': 'sum', 'another_column': 'mean'})

In [92]:
df['2021-05-24 11:00:00':'2021-05-24 13:00:00']

,data.grid_reading_kwh,data.grid_kwh_unit,data.status,data.meter_sr_no,data.error_reason,data.R_voltage,data.Y_volatge,data.B_voltage,data.r_current,data.y_current,data.b_current,site_id,location_id,data.cumm_pf,data.frequency,units
data.creation_time,,,,,,,,,,,,,,,,
2021-05-24 11:01:39,1535.0,0.0,P,DAL0000997,No Consumption,244.6,243.8,242.5,0.57,0.38,0.34,6075bb51153a20.38235471,500000010032,0.85,50.1,0.0
2021-05-24 11:04:54,1535.0,0.0,P,DAL0000997,No Consumption,245.0,243.6,242.5,0.58,0.62,0.34,6075bb51153a20.38235471,500000010032,0.89,50.2,0.0
2021-05-24 11:08:10,1535.0,0.0,P,DAL0000997,No Consumption,245.7,243.9,240.8,0.69,0.62,0.34,6075bb51153a20.38235471,500000010032,0.91,50.1,0.0
2021-05-24 11:11:24,1535.0,0.0,P,DAL0000997,No Consumption,245.3,243.4,240.8,0.69,0.61,0.09,6075bb51153a20.38235471,500000010032,0.86,50.1,0.0
2021-05-24 11:14:41,1535.0,0.0,P,DAL0000997,No Consumption,245.9,243.5,241.4,0.82,0.61,0.09,6075bb51153a20.38235471,500000010032,0.88,50.1,0.0
2021-05-24 11:17:56,1535.0,0.0,P,DAL0000997,No Consumption,245.1,242.9,241.6,0.84,0.63,0.09,6075bb51153a20.38235471,500000010032,0.88,50.1,0.0
2021-05-24 12:11:42,1536.0,1.0,P,DAL0000997,Success,245.5,243.8,242.3,9.39,0.32,0.48,6075bb51153a20.38235471,500000010032,1.00,50.1,1.0
2021-05-24 12:14:58,1536.0,0.0,P,DAL0000997,No Consumption,247.1,243.6,240.8,1.17,0.31,0.76,6075bb51153a20.38235471,500000010032,0.99,50.1,0.0
2021-05-24 12:18:16,1536.0,0.0,P,DAL0000997,No Consumption,246.7,244.1,241.0,0.64,0.29,0.50,6075bb51153a20.38235471,500000010032,1.00,50.1,0.0


In [67]:
df_resample=df[['units']].resample(rule='1H').sum()

In [69]:
df_resample[df_resample['units']>10]

,units
data.creation_time,
2021-05-25 10:00:00,24.0
2021-05-30 06:00:00,47.0
2022-07-05 15:00:00,32.0
2022-07-08 09:00:00,32.0


In [72]:

df_resample.isnull().sum()

units    0
dtype: int64

In [77]:
df_resample['2021-05-25 01:00:00':'2021-05-25 10:00:00'][0:50]       #'2021-05-30 06:00:00'

,units
data.creation_time,
2021-05-25 01:00:00,0.0
2021-05-25 02:00:00,0.0
2021-05-25 03:00:00,0.0
2021-05-25 04:00:00,0.0
2021-05-25 05:00:00,0.0
2021-05-25 06:00:00,0.0
2021-05-25 07:00:00,0.0
2021-05-25 08:00:00,0.0
2021-05-25 09:00:00,0.0


In [83]:
df_resample

,units
data.creation_time,
2021-05-01 00:00:00,3.0
2021-05-01 01:00:00,4.0
2021-05-01 02:00:00,3.0
2021-05-01 03:00:00,3.0
2021-05-01 04:00:00,4.0
...,...
2023-08-17 19:00:00,0.0
2023-08-17 20:00:00,1.0
2023-08-17 21:00:00,2.0


In [66]:
df['units'].unique()

array([ 0.,  1., 24.,  5., 47.,  2.,  4.,  3.,  9., 32.,  6.,  8., 10.])

In [48]:
a=df[df['data.grid_kwh_unit'] != df['units']] 

In [49]:
a[['data.grid_kwh_unit','units']].info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 9 entries, 2022-06-21 16:56:13 to 2023-04-01 01:58:39
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   data.grid_kwh_unit  9 non-null      float64
 1   units               9 non-null      float64
dtypes: float64(2)
memory usage: 216.0 bytes


In [50]:
a[['data.grid_kwh_unit','units']]

,data.grid_kwh_unit,units
data.creation_time,,
2022-06-21 16:56:13,0.00,1.0
2022-10-01 00:14:08,0.06,1.0
2022-11-01 02:35:50,0.31,1.0
2022-12-04 07:18:06,0.12,1.0
2023-01-01 01:18:38,0.71,1.0
2023-02-01 06:53:06,0.18,1.0
2023-02-18 19:30:04,2.00,3.0
2023-03-01 12:35:55,0.74,1.0
2023-04-01 01:58:39,0.21,1.0


In [35]:
a['data.grid_kwh_unit'][4]

0.03

In [52]:
a['units'].unique()

array([1., 3.])

In [19]:
a['2022-08-31 21:00:37':'2022-09-30 22:10:42']

,data.grid_reading_kwh,data.grid_kwh_unit,data.status,data.meter_sr_no,data.error_reason,data.R_voltage,data.Y_volatge,data.B_voltage,data.r_current,data.y_current,data.b_current,site_id,location_id,data.cumm_pf,data.frequency,units
data.creation_time,,,,,,,,,,,,,,,,
2022-08-31 21:00:37,15573.78,0.78,P,DAL0000997,Success,233.0,228.7,230.0,0.44,0.72,1.17,6075bb51153a20.38235471,500000010032,0.99,50.2,0.78
2022-08-31 21:04:57,15573.82,0.04,P,DAL0000997,Success,233.9,226.1,229.1,0.44,0.76,1.17,6075bb51153a20.38235471,500000010032,0.99,50.1,0.04
2022-08-31 21:09:11,15573.84,0.02,P,DAL0000997,Success,233.9,226.5,229.0,0.44,0.75,0.57,6075bb51153a20.38235471,500000010032,0.98,50.1,0.02
2022-08-31 21:13:34,15573.87,0.03,P,DAL0000997,Success,233.4,228.2,228.5,0.44,0.76,0.96,6075bb51153a20.38235471,500000010032,0.99,50.2,0.03
2022-08-31 21:17:50,15573.90,0.03,P,DAL0000997,Success,231.4,229.3,229.2,0.44,0.74,0.66,6075bb51153a20.38235471,500000010032,0.99,50.1,0.03
2022-08-31 21:22:03,15574.03,0.13,P,DAL0000997,Success,231.8,228.2,227.3,0.44,0.75,8.12,6075bb51153a20.38235471,500000010032,0.99,50.1,0.13
2022-08-31 21:26:20,15574.18,0.15,P,DAL0000997,Success,232.1,228.6,227.0,0.44,0.42,8.08,6075bb51153a20.38235471,500000010032,0.99,50.2,0.15
2022-08-31 21:30:34,15574.32,0.14,P,DAL0000997,Success,232.1,227.4,225.9,0.44,0.75,7.94,6075bb51153a20.38235471,500000010032,0.99,50.2,0.14
2022-08-31 21:34:52,15574.47,0.15,P,DAL0000997,Success,231.8,225.8,225.1,0.44,0.72,8.04,6075bb51153a20.38235471,500000010032,0.99,50.1,0.15


In [24]:
a['2022-09-30 23:44:18':'2022-10-01 01:01:22']

,data.grid_reading_kwh,data.grid_kwh_unit,data.status,data.meter_sr_no,data.error_reason,data.R_voltage,data.Y_volatge,data.B_voltage,data.r_current,data.y_current,data.b_current,site_id,location_id,data.cumm_pf,data.frequency,units
data.creation_time,,,,,,,,,,,,,,,,
2022-09-30 23:44:18,16348.86,0.02,P,DAL0000997,Success,240.4,238.6,237.7,0.66,0.44,0.57,6075bb51153a20.38235471,500000010032,0.99,50.0,0.02
2022-09-30 23:48:32,16348.89,0.03,P,DAL0000997,Success,241.2,239.3,235.9,0.81,0.44,0.57,6075bb51153a20.38235471,500000010032,0.99,50.2,0.03
2022-09-30 23:57:08,16348.94,0.05,P,DAL0000997,Success,242.3,239.9,236.3,0.66,0.44,0.34,6075bb51153a20.38235471,500000010032,0.99,50.1,0.05
2022-10-01 00:01:22,16348.00,0.00,P,DAL0000997,No Consumption,242.3,239.9,236.3,0.66,0.44,0.34,6075bb51153a20.38235471,500000010032,0.99,50.1,-0.94
2022-10-01 00:14:08,16349.00,0.06,P,DAL0000997,Success,243.2,238.1,238.1,0.62,0.35,0.39,6075bb51153a20.38235471,500000010032,0.97,50.1,1.00


In [27]:
# a[['units']<0]
a[a['units']<0]

,data.grid_reading_kwh,data.grid_kwh_unit,data.status,data.meter_sr_no,data.error_reason,data.R_voltage,data.Y_volatge,data.B_voltage,data.r_current,data.y_current,data.b_current,site_id,location_id,data.cumm_pf,data.frequency,units
data.creation_time,,,,,,,,,,,,,,,,
2022-10-01 00:01:22,16348.0,0.0,P,DAL0000997,No Consumption,242.3,239.9,236.3,0.66,0.44,0.34,6075bb51153a20.38235471,500000010032,0.99,50.1,-0.94
2022-11-01 00:02:55,16597.0,0.0,P,DAL0000997,No Consumption,254.4,249.0,252.0,0.04,0.29,0.04,6075bb51153a20.38235471,500000010032,0.84,50.2,-0.69
2022-12-01 00:00:50,16758.0,0.0,P,DAL0000997,No Consumption,254.7,252.1,254.8,0.00,0.00,0.00,6075bb51153a20.38235471,500000010032,1.00,50.2,-0.88
2023-01-01 00:15:43,17046.0,0.0,P,DAL0000997,No Consumption,256.5,251.1,253.0,0.09,0.00,0.41,6075bb51153a20.38235471,500000010032,0.99,50.0,-0.29
2023-02-01 00:00:18,17514.0,0.0,P,DAL0000997,No Consumption,255.8,251.7,255.2,0.00,0.01,0.29,6075bb51153a20.38235471,500000010032,0.97,50.2,-0.82
2023-03-01 00:03:55,17697.0,0.0,P,DAL0000997,No Consumption,256.9,253.3,255.3,0.03,0.07,0.14,6075bb51153a20.38235471,500000010032,0.99,50.4,-0.26
2023-04-01 00:06:43,17884.0,0.0,P,DAL0000997,No Consumption,252.1,247.8,249.9,0.03,0.18,0.14,6075bb51153a20.38235471,500000010032,0.86,50.2,-0.79


In [ ]:
df_units=df[['data.grid_kwh_unit','units']]

In [ ]:
df_units.dropna(inplace=True)

In [ ]:
# df_units['2021-05-24 16:00:00':'2021-05-25 12:00:00']

In [ ]:
df

In [ ]:
df[df['data.grid_kwh_unit']>10]

In [ ]:
# df[df['data.grid_kwh_unit','units']['2022-07-06 02:31:49':'2022-07-06 19:51:15']][0:50]

In [ ]:
df=final_lst[0]

In [ ]:
df['2021-08-17 20:00:00':'2021-08-18 02:00:00'][0:50]

In [ ]:
df['units']=df['data.grid_reading_kwh']-df['data.grid_reading_kwh'].shift(1)

In [ ]:
df['2021-08-17 20:00:00':'2021-08-18 02:00:00'][0:50]

In [ ]:
df_min=df[['data.grid_reading_kwh']].resample(rule='1H').min()
df_min

In [ ]:
df_resampled=df[['data.grid_reading_kwh']].resample(rule='1H').max()
df_resampled

In [ ]:
df_resampled['units']=df_resampled['data.grid_reading_kwh']-df_min['data.grid_reading_kwh']
df_resampled